In [10]:
# Note: need to run the following in powershell
#       pip install polygon-api-client

import pandas as pd
from polygon import RESTClient

# Note: the furthest back we can go with the free plan seems to be Feb 12, 2023
key = 'API KEY HERE'
client = RESTClient(key)

ModuleNotFoundError: No module named 'polygon'

In [ ]:
def multiple_stock_table(date):
    """
    Returns a dataframe containing all info on all stocks at a single specified date
    Date is a string of the form YYYY-MM-DD
    """
    
    data = client.get_grouped_daily_aggs(date)

    df = pd.DataFrame({
        'Close' : [stock.close for stock in data],
        'Volume': [stock.volume for stock in data],
        'Open' : [stock.open for stock in data],
        'High' : [stock.high for stock in data],
        'Low' : [stock.low for stock in data], 
        'Vwap': [stock.vwap for stock in data],
        'Transactions': [stock.transactions for stock in data]},
        index = [stock.ticker for stock in data])
    
    df = df.sort_values('Volume', ascending = False)
    
    return df


In [ ]:
df = multiple_stock_table('2023-02-13')
df

,Close,Volume,Open,High,Low,Vwap,Transactions
NVDA,21.7880,474918620.0,21.5380,22.0480,20.9620,21.6940,463567.0
SRNE,0.2572,297283013.0,0.6053,0.6600,0.2388,0.3656,227220.0
TSLA,194.6400,172475452.0,194.4150,196.3000,187.6100,193.2998,1522129.0
TQQQ,24.9500,139869852.0,24.1400,25.1000,23.8500,24.6580,296346.0
PLTR,7.6100,87397065.0,7.5800,7.7750,7.4500,8.1066,204947.0
...,...,...,...,...,...,...,...
RNWZ,24.2266,0.0,24.2266,24.2266,24.2266,NaN,NaN
MNBD,25.7430,0.0,25.7430,25.7430,25.7430,NaN,NaN
ESCR,18.2850,0.0,18.2850,18.2850,18.2850,NaN,NaN
CSH,25.4300,0.0,25.4300,25.4300,25.4300,NaN,NaN


In [11]:
df.reset_index(inplace=True)  # Moves stock names from index to a column
df.rename(columns={"index": "Ticker"}, inplace=True)  # Rename the new column
df = df.loc[:, ~df.columns.duplicated()]

def compute_macd(df, short_window=12, long_window=26, signal_window=9):
    '''
    Takes in a dataframe from a certain date that was specified date and 
    calculates the MACD(Moving Average Convergence Divergence) and the signal line

    If the MACD is higher than the signal line there is a upward trend and should think of buying 
    If the MACD is lower than the signal line there is a downwards trend and should think of selling
    '''
    df = df.copy()
    df["EMA12"] = df["Close"].ewm(span=short_window, adjust=False).mean()
    df["EMA26"] = df["Close"].ewm(span=long_window, adjust=False).mean()
    df["MACD"] = df["EMA12"] - df["EMA26"]
    df["Signal_Line"] = df["MACD"].ewm(span=signal_window, adjust=False).mean()
    return df[["Ticker", "MACD", "Signal_Line"]]

macd_df = compute_macd(df)

macd_df

NameError: name 'df' is not defined

In [3]:
from datetime import datetime

# Note:  the API lists timestamps rather than dates
#        these are unix millisecond timestamps, which describe the number of milliseconds 
#        since Jan 1, 1970, midnight UTC/GMT.
#        We use the following function to convert timestamps to date strings

def timestamp_to_date(timestamp):
    """
    Convert a UNIX timestamp in milliseconds to a date string in the format YYYY-MM-DD.
    """
    
    # Convert to seconds and shift by a day
    epoch_secs = timestamp/1000 + 86400
    
    # Convert the timestamp to a datetime object
    dt_object = datetime.fromtimestamp(epoch_secs)
    
    # Convert to string of form YYYY-MM-DD
    date_string = dt_object.strftime('%Y-%m-%d')
    
    return date_string

In [4]:
def single_stock_table(ticker, start_date, end_date):
    """
    Returns a dataframe containing all info on a given stock from start_date to end_date
    Dates are strings of the form YYYY-MM-DD
    """
    
    data = client.get_aggs(ticker, 1, 'day', start_date, end_date)
    
    df = pd.DataFrame({
    'Close' : [entry.close for entry in data],
    'Volume': [entry.volume for entry in data],
    'Open' : [entry.open for entry in data],
    'High' : [entry.high for entry in data],
    'Low' : [entry.low for entry in data], 
    'Vwap': [entry.vwap for entry in data],
    'Transactions': [entry.transactions for entry in data]},
    index = [timestamp_to_date(entry.timestamp) for entry in data]
    )
    
    return df



In [5]:
df = single_stock_table('NVDA', '2022-08-01', '2025-02-09')
df

NameError: name 'client' is not defined

In [6]:
data = client.get_aggs('NVDA', 1, 'day','2021-08-01', '2025-02-09')
data

NameError: name 'client' is not defined

In [ ]:
import requests  # <-- Added this line

def fetch_macd(stock_ticker, timespan="day", short_window=12, long_window=26, signal_window=9, limit=10):
    """
    Fetch MACD data from Polygon API for a given stock.

    If the MACD is higher than the signal line there is a upward trend and should think of buying 
    If the MACD is lower than the signal line there is a downwards trend and should think of selling
    """
    url = f"https://api.polygon.io/v1/indicators/macd/{stock_ticker}?timespan={timespan}&adjusted=true&short_window={short_window}&long_window={long_window}&signal_window={signal_window}&series_type=close&order=desc&limit={limit}&apiKey={key}"
    
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching MACD data: {response.json()}")

    data = response.json()
    
    if "results" not in data or "values" not in data["results"]:
        raise Exception("No MACD data available")

    macd_values = data["results"]["values"]
    df = pd.DataFrame(macd_values)[["timestamp", "value", "signal"]]
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.rename(columns={"value": "MACD", "signal": "Signal_Line"}, inplace=True)
    
    return df


macd_df = fetch_macd("AAPL")
macd_df

NameError: name 'df' is not defined

In [ ]:
def stock_proc(ticker, start_date, end_date, n):
    """
    Fetch stock data for a given ticker between start_date and end_date,
    and compute the Price Rate of Change (PROC) over the last 'n' days.
    Automatically fixes incorrect date order.
    """
    # Ensure correct date order
    start_date, end_date = min(start_date, end_date), max(start_date, end_date)

    # Fetch stock data from API
    data = client.get_aggs(ticker, 1, 'day', start_date, end_date)
    
    # Convert data into a DataFrame
    df = pd.DataFrame({
        'Close': [entry.close for entry in data],
        'Volume': [entry.volume for entry in data],
        'Open': [entry.open for entry in data],
        'High': [entry.high for entry in data],
        'Low': [entry.low for entry in data], 
        'Vwap': [entry.vwap for entry in data],
        'Transactions': [entry.transactions for entry in data]
    }, index=[timestamp_to_date(entry.timestamp) for entry in data])

    # Ensure data is sorted by date
    df.sort_index(inplace=True)
    
    # Compute PROC
    df["Close_n_days_ago"] = df["Close"].shift(n)  # Closing price 'n' days ago
    df["PROC"] = ((df["Close"] - df["Close_n_days_ago"]) / df["Close_n_days_ago"]) * 100
    
    # Get the latest row with valid PROC
    latest_data = df.iloc[-1][["Close", "Close_n_days_ago", "PROC"]]
    
    # Return the final DataFrame
    result = pd.DataFrame({
        "Ticker": [ticker],
        "Current Close": [latest_data["Close"]],
        f"Close {n} Days Ago": [latest_data["Close_n_days_ago"]],
        f"PROC ({n} days)": [latest_data["PROC"]]
    })
    
    return result


In [ ]:
df = stock_proc("NVDA", "2025-02-09", "2025-01-10", 10)
df